In [1]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets

## Reading Data

In [2]:
# Data Setting
TEXT = data.Field(batch_first = True,
                  fix_length = 500,
                  tokenize=str.split,
                  pad_first=True,
                  pad_token='[PAD]',
                  unk_token='[UNK]')

LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(text_field = TEXT, 
                                             label_field = LABEL)

In [3]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}')
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 25000
Test Data Length : 25000


In [4]:
# Data Fields
train_data.fields

{'text': <torchtext.data.field.Field at 0x16192f546d8>,
 'label': <torchtext.data.field.LabelField at 0x16192f546a0>}

In [5]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the str

## Pre-processing Data

In [6]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

In [7]:
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
    
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

## Making Vocab & Setting Embedding

In [8]:
model_config = {'emb_type' : 'fasttext', 'emb_dim' : 300}

In [9]:
# fasttext pre-trained 

# TEXT.build_vocab(train_data,
#                  min_freq = 2, 
#                  max_size = None,
#                  vectors = 'fasttext.en.300d')

# 위와 같은 옵션으로 fasttext Pre-Trained vector를 가져올 수 있으나 현재 버그가 있어서 직접 다운받아 사용하는 방법으로 변경하였습니다.
# ISSUE : https://github.com/pytorch/text/issues/634

# 아래 코드는 jupyter notebook에서만 실행됩니다. python script로 실행할 경우에는 따로 다운받아서 준비해주시기 바랍니다.
! wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec' 

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [9]:
# fasttext pre-trained 
from torchtext.vocab import Vectors
fasttext_vectors = Vectors('./wiki.en.vec')

TEXT.build_vocab(train_data,
                 min_freq = 2, 
                 max_size = None,
                 vectors = fasttext_vectors)

LABEL.build_vocab(train_data)

model_config['vocab_size'] = len(TEXT.vocab)

100%|██████████████████████████████████████████████████████████████████████| 2519370/2519370 [05:10<00:00, 8123.26it/s]


In [10]:
# Vocabulary Info
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
    if idx >= 10:
        break    
    print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Vocab Size : 51956
Vocab Examples : 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 and 3
	 a 4
	 of 5
	 to 6
	 is 7
	 in 8
	 it 9
---------------------------------
Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


In [11]:
# Check embedding vectors
TEXT.vocab.vectors.shape

torch.Size([51956, 300])

## Spliting Validation Data & Making Data Iterator

In [12]:
# Spliting Valid set
train_data, valid_data = train_data.split(random_state = random.seed(0),
                                          split_ratio=0.8)

In [13]:
model_config['batch_size'] = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=model_config['batch_size'],
    device=device)

In [14]:
# Check batch data
sample_for_check = next(iter(train_iterator))
print(sample_for_check)
print(sample_for_check.text)
print(sample_for_check.label)


[torchtext.data.batch.Batch of size 30]
	[.text]:[torch.cuda.LongTensor of size 30x500 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 30 (GPU 0)]
tensor([[    1,     1,     1,  ...,  1262,    22,   119],
        [    1,     1,     1,  ...,  5769,     3,  4838],
        [    1,     1,     1,  ...,  3035,    76,  4462],
        ...,
        [    1,     1,     1,  ..., 15469,  5258,     0],
        [    1,     1,     1,  ...,    16,   916,   467],
        [    1,     1,     1,  ...,    24,   233,  2630]], device='cuda:0')
tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')


In [15]:
# Check reverting data
print(' '.join([TEXT.vocab.itos[int(x)] for x in sample_for_check.text[0,:] if x not in [0,1]]))
print(LABEL.vocab.itos[int(sample_for_check.label[0])]) 

i will never forget the wit and great comedy of the original vacation movie the lines pacing and timing of events in that film are outstanding however this european vacation sequel is a major let down in this sequel the griswalds win a european vacation on a game show the problem is that many of the jokes in the film are little more than mild ha ha laughs for example a flight attendant on an airplane asks clark do you want your coke in the can clark answers back no i'll have it right here that's really about the only line that is funny in this film european vacation's humor is strained as if the writers borrowed all the jokes from the first movie tried to re hash a script that had been done before and relied on a ridiculous slap stick chase scene sequence toward the end of the picture just to kill time worse the natural comic standouts like randy quaid as cousin eddie and the original kids who played rusty and audrey from the first movie so well are nowhere to be found their replacemen

## Making Model

In [16]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
        else:
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        self.RNN = nn.RNN (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction,
                            model_config['output_dim'])
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) 
        # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) 
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        
        # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction) 
        # hidden : (Batch_Size, num_direction, Hidden_dim)
        
        last_output = output[:,-1,:]

        # last_output : (Batch_Size, Hidden_dim * num_direction)
        return self.fc(self.drop(last_output))

### Checking feed-forward

In [17]:
model_config.update(dict(batch_first = True,
                         model_type = 'RNN',
                         bidirectional = True,
                         hidden_dim = 128,
                         output_dim = 1,
                         dropout = 0))

In [18]:
model = SentenceClassification(**model_config).to(device)

In [19]:
predictions = model.forward(sample_for_check.text).squeeze()

In [20]:
loss_fn = nn.BCEWithLogitsLoss().to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum()/len(correct)
    return acc

In [21]:
loss = loss_fn(predictions, sample_for_check.label)
acc = binary_accuracy(predictions, sample_for_check.label)

In [22]:
print(predictions)
print(loss, acc)

tensor([-0.2010,  0.0449, -0.0620,  0.0929, -0.0928, -0.1756, -0.1031,  0.0097,
         0.0383,  0.0152,  0.0135,  0.0266,  0.0342,  0.0347, -0.0694,  0.0065,
        -0.0299,  0.0539, -0.1352,  0.0446,  0.1326, -0.2946, -0.0182,  0.1253,
        -0.0865, -0.0268, -0.0198, -0.1298, -0.0785, -0.2052], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.4333, device='cuda:0')


## Training

In [23]:

def train(model, iterator, optimizer, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    batch_size = model_params['batch_size']

    for idx, batch in enumerate(iterator):
        
        # Initializing
        optimizer.zero_grad()
        
        # Forward 
        predictions = model(batch.text).squeeze()
        loss = loss_fn(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )

        # Backward 
        loss.backward()
        optimizer.step()
        
        # Update Epoch Performance
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator) 

In [24]:
def evaluate(model, iterator, loss_fn):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # evaluation mode
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### bi-RNN

In [25]:
model_config['model_type'] = 'RNN'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [26]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-RNN_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.7976  Acc : 0.7667
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6087 | Train Acc : 0.6679
	 Epoch : 0 | Valid Loss : 0.5982 | Valid Acc : 0.7553
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.6913  Acc : 0.7677
	 Epoch : 1 | Train Loss : 0.5828 | Train Acc : 0.6921
	 Epoch : 1 | Valid Loss : 0.6419 | Valid Acc : 0.6272
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.4492  Acc : 0.8333
	 Epoch : 2 | Train Loss : 0.491 | Train Acc : 0.7581
	 Epoch : 2 | Valid Loss : 0.6618 | Valid Acc : 0.6348
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.5236  Acc : 0.6333
	 Epoch : 3 | Train Loss : 0.408 | Train Acc : 0.8095
	 Epoch : 3 | Valid Loss : 0.6624 | Valid Acc : 0.6599
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.6261  Acc : 0.6667
	 Epoch : 4 | Train Loss : 0.4555 | Train Acc : 0.7732
	 Epoch : 4 | Valid Loss : 0.70

In [27]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.5872 | Test Acc : 0.7556


### bi-LSTM

In [28]:
model_config['model_type'] = 'LSTM'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [29]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-LSTM_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.7057  Acc : 0.6333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6284 | Train Acc : 0.6418
	 Epoch : 0 | Valid Loss : 0.5823 | Valid Acc : 0.684
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.4286  Acc : 0.8667
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.398 | Train Acc : 0.8256
	 Epoch : 1 | Valid Loss : 0.4884 | Valid Acc : 0.7682
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.1819  Acc : 0.93337
	 Saved at 2-epoch
	 Epoch : 2 | Train Loss : 0.3195 | Train Acc : 0.8667
	 Epoch : 2 | Valid Loss : 0.4123 | Valid Acc : 0.8377
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.07073  Acc : 1.0337
	 Saved at 3-epoch
	 Epoch : 3 | Train Loss : 0.1666 | Train Acc : 0.935
	 Epoch : 3 | Valid Loss : 0.3644 | Valid Acc : 0.846
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.09691  Acc : 0.9667
	 Epoch : 4 | Train Loss : 

In [30]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.4138 | Test Acc : 0.8287


### bi-GRU

In [31]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [32]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-GRU_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.1196  Acc : 0.9333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.3923 | Train Acc : 0.8177
	 Epoch : 0 | Valid Loss : 0.2423 | Valid Acc : 0.9042
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.09375  Acc : 0.9667
	 Epoch : 1 | Train Loss : 0.1549 | Train Acc : 0.943
	 Epoch : 1 | Valid Loss : 0.2489 | Valid Acc : 0.9046
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.02705  Acc : 1.0077
	 Epoch : 2 | Train Loss : 0.04529 | Train Acc : 0.9857
	 Epoch : 2 | Valid Loss : 0.3513 | Valid Acc : 0.8914
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.001555  Acc : 1.007
	 Epoch : 3 | Train Loss : 0.0133 | Train Acc : 0.9958
	 Epoch : 3 | Valid Loss : 0.4598 | Valid Acc : 0.8913
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.0002414  Acc : 1.07
	 Epoch : 4 | Train Loss : 0.004864 | Train Acc : 0.9988
	 Epoch : 4 | Valid Los

In [33]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.2451 | Test Acc : 0.8997


## Inference

In [34]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
model.load_state_dict(torch.load(f"./{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}.pt"))

<All keys matched successfully>

In [35]:
def predict_sentiment(model, sentence):
    model.eval()
    indexed = TEXT.numericalize(TEXT.pad([TEXT.tokenize(PreProcessingText(sentence))]))
    input_data = torch.LongTensor(indexed).to(device)
    prediction = torch.sigmoid(model(input_data))
    return prediction.item()

In [36]:
test_sentence = 'this movie is FUN'
predict_sentiment(model = model, sentence = test_sentence)

0.93869549036026